In [1]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from didcomm.message import Attachment, AttachmentDataJson
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

## Helper function

In [2]:
secrets_resolver = SecretsResolverDemo()

In [3]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [4]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Reading issuer OOB message

In [5]:
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
print(oob_url)

https://mediator.rootsid.cloud?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiMmYyMzMxYzYtNmZhYy00OTJhLWFjNTktYzQ1MTM2N2Y1MDA2IiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNjMUdhTk4xN0tZN1pwM25SN0Nheko3aGY0Ym1iZFB5cEE5R0I0NEc5bjZCZy5WejZNa3BVeVEyQTRWajE1a2FEVXdCZndaZVRQWXR5QmszSmN3VWh6WUE2alNEeUh3LlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwibGFiZWwiOiJNZWRpYXRvciIsImFjY2VwdCI6WyJkaWRjb21tL3YyIl19fQ


In [6]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '2f2331c6-6fac-492a-ac59-c451367f5006', 'from': 'did:peer:2.Ez6LSc1GaNN17KY7Zp3nR7CazJ7hf4bmbdPypA9GB44G9n6Bg.Vz6MkpUyQ2A4Vj15kaDUwBfwZeTPYtyBk3JcwUhzYA6jSDyHw.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'request-mediate', 'goal': 'RequestMediate', 'label': 'Mediator', 'accept': ['didcomm/v2']}}


## Prepare request-credential

The holder needs to create a did:peer to communicate with the issuer:

In [7]:
holder_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/holder")
print("Holder's DID:", holder_did)

Holder's DID: did:peer:2.Ez6LSsGeMovkB9bDB6mhsPMsfrTn7hWBm8pgzoU6UXHb12H7V.Vz6MkhNnr5r6GMfutEiABzmdyTkFBtxTHFQeNoLopqcBYJ4fm.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ob2xkZXIiLCJhIjpbImRpZGNvbW0vdjIiXX0


Also, the holder needs to provide a did:prism where the credential will be issued to.

In [8]:
holder_prism_did = "did:prism:a08f49e61bc46dc82b6bd54dd62087c25b53b4a7ef98e549ce62ee4ad3450d5c"

The following is a JSON-LD credential request that the holder will submmit to the issuer.

In [9]:
credential_request = {
    "credential": {
        "@context": 
        [
            "https://www.w3.org/2018/credentials/v1",
            "https://www.w3.org/2018/credentials/examples/v1"
        ],
        "id": str(uuid.uuid4()),
        "type": ["VerifiableCredential", "UniversityDegreeCredential"],
        "issuer": "TBD",
        "issuanceDate": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "credentialSubject": 
            {
              "id": holder_prism_did,
              "degree": 
                {
                    "type": "BachelorDegree",
                    "name": "Bachelor of Science and Arts"
                  }
            },
        "options": {
            "proofType": "EcdsaSecp256k1Signature2019"
        }
    }
}

Finally the holder prepares the didcomm message following issue-credential/3.0 protocol:

In [10]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/issue-credential/3.0/request-credential",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body = {
        "goal_code": "issue-credential",
        "comment": "some comment"
    },
    attachments = [
        Attachment(
                id=str(uuid.uuid4()),
                media_type= "application/json",
                format= "aries/ld-proof-vc-detail@v1.0",
                data=AttachmentDataJson(json=credential_request)
                )
    ]
                        
)

In [12]:
holder_response_message.as_dict()

{'id': 'd5a056aa-b3e7-47db-befc-c963a8e0178e',
 'type': 'https://didcomm.org/issue-credential/3.0/request-credential',
 'body': {'goal_code': 'issue-credential', 'comment': 'some comment'},
 'to': ['did:peer:2.Ez6LSc1GaNN17KY7Zp3nR7CazJ7hf4bmbdPypA9GB44G9n6Bg.Vz6MkpUyQ2A4Vj15kaDUwBfwZeTPYtyBk3JcwUhzYA6jSDyHw.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0'],
 'attachments': [{'data': {'json': {'credential': {'@context': ['https://www.w3.org/2018/credentials/v1',
       'https://www.w3.org/2018/credentials/examples/v1'],
      'id': '0c936254-289f-4c73-a7fc-965932faa28a',
      'type': ['VerifiableCredential', 'UniversityDegreeCredential'],
      'issuer': 'TBD',
      'issuanceDate': '2022-10-24T18:19:17Z',
      'credentialSubject': {'id': 'did:prism:a08f49e61bc46dc82b6bd54dd62087c25b53b4a7ef98e549ce62ee4ad3450d5c',
       'degree': {'type': 'BachelorDegree',
        'name': 'Bachelor of Science and Arts'}},
      'options':

In [69]:
holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Mediator

From the issuer DID obtained in the OOB QR Code, we can get the DID document:

In [70]:
issuer_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
print(issuer_did_doc)

{'id': 'did:peer:2.Ez6LSgi5Y8dCJg9SJ3PbbooN1Xf9rekY5Z5d29kSwynTVQi97.Vz6Mkm9nW2RvFs9gK2EEPaxRpxdPsDoRZjac7THdRGFnLsFjt.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'authentication': [{'id': 'did:peer:2.Ez6LSgi5Y8dCJg9SJ3PbbooN1Xf9rekY5Z5d29kSwynTVQi97.Vz6Mkm9nW2RvFs9gK2EEPaxRpxdPsDoRZjac7THdRGFnLsFjt.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0#6Mkm9nW2RvFs9gK2EEPaxRpxdPsDoRZjac7THdRGFnLsFjt', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LSgi5Y8dCJg9SJ3PbbooN1Xf9rekY5Z5d29kSwynTVQi97.Vz6Mkm9nW2RvFs9gK2EEPaxRpxdPsDoRZjac7THdRGFnLsFjt.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'publicKeyMultibase': 'z6Mkm9nW2RvFs9gK2EEPaxRpxdPsDoRZjac7THdRGFnLsFjt'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LSgi5Y8dCJg9SJ3PbbooN1Xf9rekY5Z5d29kSwynTVQi97.Vz6Mkm9nW2RvFs9gK2EEPaxRpxdPsDoRZjac

And from there get the issuer's endpoint

In [71]:
issuer_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
print(issuer_endpoint)

https://mediator.rootsid.cloud


Finally sending the request to the issuer:

In [72]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(issuer_endpoint, headers=headers, data = holder_packed_msg.packed_msg)
print(resp.json())


{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTJkWlRsZHdhbUpNUW5Sall6VkNkVmRPY0dOcldIVlZWemRvWjI1TlIzUkVVSGRMUlVGeE5uUlFNek5STGxaNk5rMXJiVlZUWW5rMlltaERNVlJoUm5wMFdXMHlSR05RZDBkTVdrdGlRemRCYUZCck56Sk1WelV6Ym5wbVVHTXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3eU1XeGFSMnhvWkVjNWVVeHVTblppTTFKNllWZFJkVmt5ZUhaa1YxRnBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGZ3dJelpNVTJkWlRsZHdhbUpNUW5Sall6VkNkVmRPY0dOcldIVlZWemRvWjI1TlIzUkVVSGRMUlVGeE5uUlFNek5SIiwiYXB2IjoiMV8zeThjMnBzTHNhOXhDTU4wWGx6OHRxY25QaF9Na0dreHBYY2xUYjFpayIsInNraWQiOiJkaWQ6cGVlcjoyLkV6NkxTZ1lOV3BqYkxCdGNjNUJ1V05wY2tYdVVXN2hnbk1HdERQd0tFQXE2dFAzM1EuVno2TWttVVNieTZiaEMxVGFGenRZbTJEY1B3R0xaS2JDN0FoUGs3MkxXNTNuemZQYy5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTZ1

### Issued Verifiable Credential Received

In [38]:
credential_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

In [39]:
print(credential_unpack_msg)

UnpackResult(message=Message(id='cb613637-afc9-4fa6-9428-91d89c135f0e', type='https://didcomm.org/issue-credential/3.0/issue-credential', body={'goal_code': 'issue-credential', 'comment': 'some comment'}, frm=None, to=None, created_time=None, expires_time=None, from_prior=FromPrior(iss='did:peer:2.Ez6LSsx2Bks6eXJkexY7imgMBWvS7wjoAe24MoAKtRs7Bdbve.Vz6MkrbPCW4vTaUi2iJas6dDfNEe94sadadLycubuM6XHiv5T.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', sub='did:peer:2.Ez6LSc6bpTGPpF1sYapg8f58TnLay7RVRWwh2kqoyo8so6EHd.Vz6Mku2uQ6oD9tpUhZ9sSeFnYRqcR1DMUtZdNbc6gG6hKSN1L.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0', aud=None, exp=None, nbf=None, iat=None, jti=None), please_ack=None, ack=None, thid=None, pthid=None, attachments=[Attachment(data=AttachmentDataJson(json={'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1'], '

In [40]:
print(credential_unpack_msg.message.attachments[0].data.json)

{'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1'], 'id': 'f814d273-9efe-4123-bd81-4b22ef52a9e6', 'type': ['VerifiableCredential', 'UniversityDegreeCredential'], 'issuer': 'did:prism:5a485cd8ba57aed595373f92e56fc79fd5de459d84ea0d1b85892fc5566648af', 'issuanceDate': '2022-08-31T21:13:12Z', 'credentialSubject': {'id': 'did:prism:a08f49e61bc46dc82b6bd54dd62087c25b53b4a7ef98e549ce62ee4ad3450d5c', 'degree': {'type': 'BachelorDegree', 'name': 'Bachelor of Science and Arts'}}, 'options': {'proofType': 'EcdsaSecp256k1Signature2019'}, 'proof': {'type': 'EcdsaSecp256k1Signature2019', 'created': '2022-08-31T21:13:25Z', 'verificationMethod': 'did:prism:5a485cd8ba57aed595373f92e56fc79fd5de459d84ea0d1b85892fc5566648af', 'proofPurpose': 'assertionMethod', 'proofValue': 'eyJpZCI6ImRpZDpwcmlzbTo1YTQ4NWNkOGJhNTdhZWQ1OTUzNzNmOTJlNTZmYzc5ZmQ1ZGU0NTlkODRlYTBkMWI4NTg5MmZjNTU2NjY0OGFmIiwia2V5SWQiOiJpc3N1aW5nMCIsImNyZWRlbnRpYWxTdWJqZWN0Ijp7Imlzc3VhbmNlRG